
## Lab 8: Supervised Classification in Google Earth Engine
## Digital Image Processing: GIS 4050/5050


In this lab, the steps for conducting a supervised classification in Google Earth Engine will be covered. Part 1 is a walkthrough of loading a single Landsat 8 image, assembling training data using current landuse/land cover data, training the classifier, and applying the classifier to the image and symbolize. Part 2 is a walkthrough for collecting classification statistics of landsat data based on the boundaries of a polygon, and Part 3 is an applied exercise.



Deliverables: 
1. An ipynb for Part 1 Walkthrough
2. A CSV table of Part 2 Walkthrough
3. A CSV and ipynb for Part 2: Your Turn
4. Answers to the five integrated questions in a separate word document 

## Basic Machine Learning with Earth Engine - Supervised Classification

## Supervised classification algorithms available in Earth Engine

Source: https://developers.google.com/earth-engine/classification

The `Classifier` package handles supervised classification by traditional ML algorithms running in Earth Engine. These classifiers include CART, RandomForest, NaiveBayes and SVM. The general workflow for classification is:

1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.
2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Estimate classification error with independent validation data.

The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. Class labels should be consecutive, integers starting from 0. If necessary, use remap() to convert class values to consecutive integers. The predictors should be numeric.

In this example we'll be applying the CART classifier (Classification And Regression Tree) with training data built from land use and land cover data. The CART Classifier is a simple, binary, multiple decision tree model that thresholds DN's and groups pixel values based on their relationship to the threshold. Binary means that the outcome is either 1 or 0, representing the different classes for class outcomes. 

![](https://i.imgur.com/vROsEiq.png)

## Part 1: Supervised Classification Walkthrough
## Install Geemap if you need to

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=2c2a492fd58d0f6e6f358fa95839917435da8a28f3aa55284bb39b7

**Question 1: We are using a corrected image from Tier-2 processing. What pre-processing steps would have been necessary (prior to classification) had we chosen to use a raw Tier-1 image?**

We would have needed to atmospherically correct the data, check the geolocation accuracy, update digital elevation modelling sources, and improve radiometric calibration.

## Step-by-step tutorial

### Import libraries

In [2]:
import ee
import geemap

### Create an interactive map

In [3]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=v5G7vjIgzinlZ4i7RfMatuEpvc_yy5MMOnkk7sAbYLI&tc=1e_ZfJpqmC77FE6Y3rf16Re6o6Tp5v4CwLE_cYAzFwg&cc=41_A6daMxf6rBcIhLPRGHG1_KdS5DKXSRVBU-VWPoXc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7suaFiMWghnCyr2DP_qyhYOowdRucsqS9h8R1rtrbsD_e8kbgb62I

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

In [4]:
Map = geemap.Map()
Mappoint = ee.Geometry.Point([-122.4439, 37.7538])
point = ee.Geometry.Point([-87.7719, 41.8799])

# Walkthrough data is LANDSAT/LC08/C01/T1_SR

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2016-01-01', '2016-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(center=[41.8799, -87.7719], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

### Check image properties

In [5]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2016-06-17'

In [6]:
image.get('CLOUD_COVER').getInfo()

0.03

**Question 2: What is the imagery date for the selected image? What is the cloudscore for the selected image? What is the purpose of checking the image properties?**

Date is 2016-06-17.
Cloudscore is 0.03.
The purpose is to make sure that the image you have is within the timeframe you want, and that the imagery isn't obscured by clouds or inclement weather.

### Make training dataset

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [7]:
# region = Map.user_roi
region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In this example, we are going to use the [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD) to create a pixel and label dataset for training


![](https://i.imgur.com/7QoRXxu.png)

**Question 3: Do you expect there to be any problems with using the NLCD landcover types as a training dataset for the selected image location?**

No I don't think so (famous last words). The NLCD is meant to cover the entire continental United States so in theory anything within American borders should be catagorized by it's landcover types.

In [8]:
Map = geemap.Map()
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())
Map.addLayer(nlcd, {}, 'NLCD')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [9]:
# Make the training dataset.
points = nlcd.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [10]:
print(points.size().getInfo())

5000


In [11]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-86.30809357805241, 41.877691157313706]}, 'id': '0', 'properties': {'landcover': 82}}


### Train the classifier

In [12]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [13]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 548, 'B2': 650, 'B3': 1023, 'B4': 1246, 'B5': 3115, 'B6': 3649, 'B7': 2640, 'landcover': 82}}


**Question 4: What data are we using as training data? Please describe.**

We're taking the NLCD points and using those as the training data. The points are color coded to the type of landcover they represent. 

### Classify the image

In [14]:
Map = geemap.Map()

# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [15]:
class_values = nlcd.get('landcover_class_values').getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [16]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [17]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [18]:
Map = geemap.Map()
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add a legend to the map

In [19]:
Map = geemap.Map()
Map.addLayer(landcover, {}, 'Land cover')
Map.add_legend(builtin_legend='NLCD')
Map.centerObject(point, 8)
Map

Map(center=[41.8799, -87.7719], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

**Question 4: What is wrong with the classification? What is one way you could improve it?**

Doesn't do the best at constratining the classification of certain areas so they stay within their proper boundaries. Merging the various points together into polygons might improve it.

### Export the result

Export the result directly to your computer:

In [20]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [23]:
#geemap.ee_export_image(landcover, filename=out_file, scale=900)

Export the result to Google Drive:

In [24]:
geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)

## Part 2 Walkthrough: Zonal Statistics of Landsat Data

In [25]:
import ee
import geemap
import os

In [26]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [27]:
Map = geemap.Map()

# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
dem_vis = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Engine DEM to map
Map.addLayer(dem, dem_vis, 'SRTM DEM')

# Add Landsat data to map
landsat = ee.Image('LE7_TOA_5YEAR/1999_2003')

landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.4
}
Map.addLayer(landsat, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [28]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_dem_stats = os.path.join(out_dir, 'dem_stats.csv')  

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(dem, states, out_dem_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/dem_stats.csv


In [29]:
out_landsat_stats = os.path.join(out_dir, 'landsat_stats.csv')  
geemap.zonal_statistics(landsat, states, out_landsat_stats, statistics_type='SUM', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/landsat_stats.csv


In [30]:
geemap.create_download_link(out_dem_stats)

In [31]:
geemap.create_download_link(out_landsat_stats)

**## Part 3: Your Turn**
Use the space below to collect and download statistics on the landuse classification you produced in Part 1.

In [32]:
# Script goes here
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_landcover_stats = os.path.join(out_dir, 'landcover.csv')  

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(landcover, states, out_landcover_stats, statistics_type='MAXIMUM', scale=1000)


Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/landcover.csv


In [33]:
geemap.create_download_link(out_landcover_stats)

**Question 5: What was the most predominant land use class in the classified image? Which classes might you merge if you were to "clean up" the result?**

Cultivated Crop. The various flavors of urban development and the scrub classes.